# Week 11 — Model Deployment & MLOps (FastAPI + MLflow)

**Goals**
- Train a small model & expose as an API with FastAPI
- Log metrics/artifacts with MLflow (local)
- Containerization pointers

## 0) Setup

In [ ]:
# !pip -q install fastapi uvicorn mlflow scikit-learn pandas pydantic
import pandas as pd, numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import mlflow, mlflow.sklearn, json, os, textwrap

print('MLflow:', mlflow.__version__)

## 1) Train a small model + log to MLflow

In [ ]:
X, y = load_iris(return_X_y=True, as_frame=True)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

with mlflow.start_run(run_name='rf_iris_demo'):
    rf = RandomForestClassifier(n_estimators=120, random_state=42).fit(Xtr, ytr)
    pred = rf.predict(Xte)
    acc = accuracy_score(yte, pred)
    mlflow.log_metric('accuracy', acc)
    mlflow.sklearn.log_model(rf, 'model')
    mlflow.log_text('\n'.join(X.columns), 'features.txt')
acc

## 2) Create a FastAPI app file and run locally

In [ ]:
app_py = '''from fastapi import FastAPI
from pydantic import BaseModel
import mlflow, mlflow.sklearn
import pandas as pd

app = FastAPI(title='Iris Classifier API')

class IrisIn(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

# Load latest model from mlruns (for demo, we load from local path you saved above)
import os, glob
# Fallback: directly load from the artifact path saved in notebook run
model = mlflow.sklearn.load_model('model')

@app.post('/predict')
def predict(inp: IrisIn):
    df = pd.DataFrame([inp.dict()])
    y = model.predict(df)[0]
    return {'prediction': int(y)}
'''
with open('fastapi_app.py','w') as f:
    f.write(app_py)

print('Created fastapi_app.py')
print('Run locally in a terminal:')
print('  uvicorn fastapi_app:app --reload --port 8000')
print('Test:')
print('  curl -X POST http://127.0.0.1:8000/predict -H "Content-Type: application/json" \\')
print('       -d "{\\"sepal_length\\":5.1, \\"sepal_width\\":3.5, \\"petal_length\\":1.4, \\"petal_width\\":0.2}"')

## 3) Notes on containerization & CI/CD

- Create a `Dockerfile` installing `requirements.txt`, copy model artifacts, expose port 8000.

- Use GitHub Actions to build & push images; deploy to Render/EC2/Cloud Run.

- Track experiments in MLflow, store artifacts in S3/GCS/Azure.